In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Activation, Embedding, LSTM
from sklearn.model_selection import train_test_split
import scipy.signal
import numpy as np
import pandas
import plotly as py
import plotly.graph_objs as go
from plotly import tools
from matplotlib.pyplot import figure
import random
from heapq import nlargest
import math

In [ ]:
model = Sequential()
model.add(LSTM(20, input_shape = (200,10), return_sequences = True))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer='RMSprop',metrics=['binary_accuracy'])
model.summary()

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

data = pandas.read_csv('./data/Learning/MyData/Turning/ut_another_classifier/data_8_9.csv')
# data = pandas.read_csv('./data/Learning/MyData/Turning/ut_another_classifier/data_9_8.csv')


X = data[['magneticHeading','gyroX','gyroY','gyroZ','Yaw','Roll','Pitch','accx','accy','accz']]

target = data['TurnStatus']

##### 5-fold corss validation

##### fold 1 
# X_train = X[:11410]
# X_test = X[11410:20650]
# y_train = target[:11410]
# y_test = target[11410:20650]
######

##### fold 2
# X_train= X[:20650]
# X_test= X[20650:31950]
# y_train= target[:20650]
# y_test= target[20650:31950]
######

##### fold 3
# X_train= X[:31950]
# X_test= X[31950:40550]
# y_train= target[:31950]
# y_test= target[31950:40550]
######

##### fold 4
# X_train = X[:40550]
# X_test = X[40550:58237]
# y_train = target[:40550]
# y_test = target[40550:58237]
######

##### fold 5
X_train = X[:58237]
X_test = X[58237:65927]
y_train = target[:58237]
y_test = target[58237:65927]




# X_train = X[:40000]
# y_train = target[:40000]

# X_test = X[40000:]
# y_test = target[40000:]



In [ ]:
def SmoothData(data):
    size = data.shape[0] 
    dim = data.shape[1]
    
    smooth_data = np.zeros((size,dim))
    
    smooth_data[0] = data[0]
    smooth_data[1] = data[1]
    smooth_data[size-1] = data[size-1]
    smooth_data[size-2] = data[size-2]
    
    for i in range(2, size-2):
        smooth_data[i] = np.average([data[i], data[i-1], data[i+1]], axis=0)
    return smooth_data

In [ ]:

X_train  = np.array(X_train)
X_test  = np.array(X_test)
y_train  = np.array(y_train)
y_test  = np.array(y_test)

X_train = SmoothData(X_train)
X_test = SmoothData(X_test)

### one-hot encoding

In [ ]:
y_train_onehot  = []
y_test_onehot = []
for i in range(len(y_train)):
    if(y_train[i]==0):
        y_train_onehot.append(np.array([1,0]))
    else:
        y_train_onehot.append(np.array([0,1]))
        
for i in range(len(y_test)):
    if(y_test[i]==0):
        y_test_onehot.append(np.array([1,0]))
    else:
        y_test_onehot.append(np.array([0,1]))
 

In [ ]:
y_train_onehot[0:5]

In [ ]:
### shuffle the train data to decorelate the training window segment
X_train_shuffle = []
y_train_shuffle = []
shuffle_index = random.sample(range(0, 57200), 572)
for i in shuffle_index:
    X_train_shuffle.append (np.array([X_train[i:i+200]]).reshape(200,10))
    y_train_shuffle.append (np.array([y_train_onehot[i:i+200]]).reshape(200,2))

In [ ]:
history = model.fit(np.array(X_train_shuffle),
                    np.array(y_train_shuffle),
                    epochs=200,
                    validation_data = (np.array([X_train[i:i+200] for i in range(57200,58000,50)]),
                                      np.array([y_train_onehot[i:i+200] for i in range(57200,58000,50)])),
                    batch_size=64)


In [ ]:
results = model.predict(np.array(X_test[:7600]).reshape(38,200,10))
results = results.reshape(7600,2)

In [ ]:
len(X_test)

In [ ]:
result_class = []
for i in results:
#     if(max(i) < 0.5):
#         result_class.append(0)
#     else:
    if(np.argmax(i)==0):
        result_class.append(0)
    else:
        result_class.append(1)

In [ ]:
"""
Utitlity Functions
"""
"""
Computing the energy (on y-axis)

First, scale the accelerometer signal to the 9.8 
Second, iterate list of events, calculate energy of the y-axis 
"""

def Energy(data):
    data = [abs(number) for number in data]
    return np.sum(np.power(data,2))

    
"""
Scaling turn signal to m/s2
"""
def Scale(data):
    scaled_data = data.copy()
    for i in range(len(scaled_data)):
        scaled_data[i] = scaled_data[i] * 9.8
    return scaled_data



In [ ]:
"""
Creating list of events and 
removing events with energy < 20 and duration lower than 30

"""

ut = 0
num_events = 0
events = []
freq = 100 
scaled_data = Scale([X_test[i][8] for i in range(len(X_test))]) # ay

for i in range(len(result_class)):
    if(result_class[i]==1):
        ut += 1
    else:
        if(ut > 100):
            signal = [abs ((scaled_data)[i]) for i in range(i-ut, i-1)]
            energy = Energy(signal)
            if(energy > 10):
                events.append(("ut", i-ut, i-1, ut/freq))
                print(num_events, "- ut event," ,"duration:", ut/freq, " tsteps", "init:", i-ut ,"end:", i-1)
                num_events += 1
            ut = 0
# add the dumb event


In [ ]:
def ComputeIntensity(signal):
    max_values = np.mean(nlargest(3, signal))
    if(max_values > 0.1 and max_values < 0.2):
        return 0
    elif(max_values > 0.2 and max_values< 0.4):
        return 1
    elif(max_values > 0.4):
        return 2  

In [ ]:
"""
Assign intensity to each event 
0, 1, 2 --> Low, Medium, High
thresholds:
    0.98 0.1G
    1.96 0.2G
    3.92 0.4G
"""    
i = 0
for e in events:
    event_type, init, end, duration = e
    signal = [abs(X_test)[i][8] for i in range(init, end)]
    events[i] += (ComputeIntensity(signal),)
    i +=1


In [ ]:
"""
merging neighbour events
"""
duration = 0
merge_index = 0
deletion_index = []
events.append(('',math.inf,math.inf,math.inf))

for i in range(0,len(events)-1):
    if(abs((events[i][2])-(events[i+1][1]))<11 and events[i][0]==events[i+1][0]):
        duration +=1
    else:
        init = events[i-duration][1]
        end = events[i][2]-1
        events.append((events[i-duration][0], init, end, end-init, events[i-duration][4]))
        deletion_index.append(np.arange(i-duration,i+1))
        duration = 0

In [ ]:
events.remove(('',math.inf,math.inf,math.inf))

In [ ]:
# deletion_index
del events[0:3]

In [ ]:
result_class = np.zeros(len(X_test))
for e in events:
    event_type, init, end, duration, intensity = e
    if(event_type!=''):
        if(event_type=="ut"):
            result_class[init:end] = 1

In [ ]:
from sklearn.metrics import confusion_matrix
import plotly.express as px


y_true = y_test.copy()
y_pred = result_class.copy()
classes = [0,1]
confusion_matrix = confusion_matrix(y_true, y_pred, labels=[0,1])
confusion_matrix
# fig = px.imshow(confusion_matrix)

# fig.show()

### Graph the result

In [ ]:

fig =tools.make_subplots(rows=1,cols=1)

result =  go.Scatter(
    x = list(range(0,7600)),
    y = result_class,
    mode = 'lines',
    name='model_result'
)

test =  go.Scatter(
    x = list(range(0,7600)),
    y = y_test,
    mode = 'lines',
    name='test'
)

fig.append_trace(result,1,1)
fig.append_trace(test,1,1)


fig['layout'].update(height=500, title = 'Turning Model Classification')


py.offline.plot(fig, filename='fold5_cross_validation_uturn')


### Performance Calculator

In [ ]:
"""
Precision = True predicted turnings / total turnings
Recall = True predicted turnings / total predicted turnings

F1 = ( 2 . precision . recall ) / precision + recall
performance = F1 * (IOU / #turning)
"""

fold1_precision = 3 / 3
fold1_recall = 4 / 6 
fold1_f1 = (2 * fold1_precision * fold1_recall) / (fold1_precision + fold1_recall)
fold1_iou = (112.0/807 +  719.0/3243 + 138.0/1129 + 109.0/1129) / 3
fold1_perf = fold1_f1 * fold1_iou
########
fold2_precision = (2 / 3)
fold2_recall = (2 / 2 )
fold2_f1 = (2 * fold2_precision * fold2_recall) / (fold2_precision + fold2_recall)
fold2_iou = (718.0/928 + 553.0/730) / 3
fold2_perf = fold2_f1 * fold2_iou
########
fold3_precision = (3 / 3)
fold3_recall = (3 / 3 )
fold3_f1 = (2 * fold3_precision * fold3_recall) / (fold3_precision + fold3_recall)
fold3_iou = (647.0/799 + 620.0/755 + 830.0/866) /3
fold3_perf = fold3_f1 * fold3_iou
########
fold4_precision = (2 / 3)
fold4_recall = (2 / 4 )
fold4_f1 = (2 * fold4_precision * fold4_recall) / (fold4_precision + fold4_recall)
fold4_iou = (605.0/ 705 + 264.0/1138) /2 
fold4_perf = fold4_f1 * fold4_iou
########
fold5_precision = (2 / 3)
fold5_recall = (2 / 3 )
fold5_f1 = (2 * fold5_precision * fold5_recall) / (fold5_precision + fold5_recall)
fold5_iou = (403.0/ 1018)  
fold5_perf = fold5_f1 * fold5_iou
########

print(fold1_perf,fold2_perf,fold3_perf,fold4_perf,fold5_perf)
print("performance mean:", np.mean([fold1_perf, fold2_perf, fold3_perf, fold4_perf, fold5_perf]))
print("f1-score mean:", np.mean([fold1_f1,fold2_f1,fold3_f1,fold4_f1,fold5_f1]))
print("iou mean:", np.mean([fold1_iou,fold2_iou, fold3_iou,fold4_iou,fold5_iou]))

### Save model

In [ ]:
from keras.models import model_from_json
import os

# serialize model to JSON
model_json = model.to_json()
with open("./Learning_model/Turning/ut_cv/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("./Learning_model/Turning/ut_cv/model.h5")
print("Saved model to disk")

### Load model

In [ ]:
from keras.models import model_from_json
import os

# load json and create model
json_file = open('./Learning_model/Turning/ut_cv/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./Learning_model/Turning/ut_cv/model.h5")
print("Loaded model from disk")
model = loaded_model